**Problem Statement**

Histopathology is the study of diseased cells using a microscope. We will be using pictures taken by histopathologists of cancerous and non-cancerous lymph nodes cells, cleaning and preprocessing those images, and subsequently building a Convolutional Neural Network, or CNN, architecture in an attempt to perform binary classification of each photo as either metastatic cancer or a healthy tissue sample. Our first steps should be to read in the data and explore its structure and volume.

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

We are given a directory of images called test, a directory of images called train, a sample submission .csv, and a .csv of the labels for the training data. Let's start checking out what's inside each of them.

In [ ]:
base_dir = '/kaggle/input/histopathologic-cancer-detection/'
train_dir = base_dir + 'train'
test_dir = base_dir + 'test'

In [ ]:
submission = pd.read_csv(base_dir + "sample_submission.csv")

In [ ]:
labels = pd.read_csv(base_dir + 'train_labels.csv')
labels.head()

In [ ]:
len(labels)

We see that the labels file contains just the id of the photo from the training set, and the class label of either a 0 for healthy cells or a 1 for metastatic cancer present, for a total size of 220025 rows of 2 columns.

In [ ]:
f"The ratio of regular pictures to cancer pictures is {round(labels['label'].value_counts()[0] / labels['label'].value_counts()[1], 2)}"

There are about 50% more regular pictures than cancerous picture, which means the class split is about 60/40. Fortunately the classes are not ridiculously imbalanced to the point where we would see problems. Often times in datasets dealing with fraud or disease detection, the positive case is so rare that models will learn to simply guess the negative case every time to max out accuracy easily.

In [ ]:
sns.set_theme(palette="pastel")
sns.countplot(labels, x = 'label')
plt.xticks(range(2), ['Regular', 'Cancer'])
plt.xlabel("Labels")
plt.ylabel("Count")
plt.title("Label Counts")
plt.show()

Next, we can open up some of the images from the training set and pair them with their matching label from the labels file.

In [ ]:
from PIL import Image

fig = plt.figure(figsize = (8, 8))
train_files = os.listdir(base_dir + 'train')

for i, pic in enumerate(train_files[0:25]):
    ax = fig.add_subplot(5, 5, i + 1)
    img = Image.open(base_dir + 'train/' + pic)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    
    label = labels.loc[labels["id"] == pic.split('.')[0], 'label'].values[0]
    ax.set_title(f"{['Cancer' if label == 1 else 'Regular'][0]}")

While the images obviously look quite different, to the untrained human eye with no medical expertise, it would be quite difficult to differentiate which images present with cancerous cells. Hopefully our model is able to teach itself how to tell them apart much faster than humans learn to tell them apart!

In [ ]:
fig = plt.figure(figsize = (8, 8))
test_files = os.listdir(base_dir + 'test')

for i, pic in enumerate(test_files[0:25]):
    ax = fig.add_subplot(5, 5, i + 1)
    img = Image.open(base_dir + 'test/' + pic)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])

The test set images look fairly similar, with the obvious exception of the labels, which are unavailable for us to use.

In [ ]:
sample = Image.open(base_dir + 'test/' + test_files[0])

The individual images are 96x96 pixels, for a total of 9216 pieces of information per picture. We would like to read in the training images with the powerful keras function flow_from_directory, but to do that we must first preprocess the file structure. Currently, all of the images of both classes are stored in the same directory, but we need to create subdirectories for each class or the keras function will be confused.

In [ ]:
import shutil

os.makedirs(os.path.join('train', 'regular'), exist_ok=True)
os.makedirs(os.path.join('train', 'cancer'), exist_ok=True)

for index, row in labels.iterrows():
    print(index)
    image_id = row['id']
    decision = row['label']
    source_path = os.path.join('train', f'{image_id}.tif')
    
    if decision == 0:
        destination_path = os.path.join('train', 'regular', f'{image_id}.tif')
    else:
        destination_path = os.path.join('train', 'cancer', f'{image_id}.tif')
        
    shutil.move(source_path, destination_path)

Now that our file system is rearranged properly, we can work on the preprocessing of the actual data and modeling of the CNN model architecture.

**Preprocessing**

When dealing with image data, we will typically want to do several steps prior to the training and learning stages to ensure the best possible results in a standardized procedure:

* Convert the image data for each file into a 1-D list so that our CNN model can ingest them easily. 
* Normalize the pixel values to a decimal value between 0 and 1, typically by dividing the RGB values by 255
* Artifically boost the size of the dataset by taking images and adding in variants of the original, with some transformation done such as flipping, rotating, shearing, etc.

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Break up data into training and validation set
train, val = train_test_split(labels, stratify = labels.label, test_size = 0.2)
print(len(train), len(val))

In [ ]:
# Define hyperparameters + parameters
img_sz = (96, 96)
classes = 2
batch_sz = 32
epochs = 20
learning_rate = 0.005

In [ ]:
# Normalize both the train and test data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
    # Add in preprocessing transformations                               
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
    # Add in preprocessing transformations
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest')

In [ ]:
# Pull the data from each subdirectory into the ImageDataGenerator function
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/histopathologic-cancer-detection/',
    classes = ['cancer', 'regular'],
    color_mode = 'rgb',
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = True)

**Modeling**

Our CNN has four convolutional layers, three max pooling layers, and two dense layers. The first convolutional layer has 32 filters with a kernel size of 3x3 and a ReLU activation function. The input shape is height and width of the images (96x96) and 3 channels for RGB. Then our first max pooling layer has a pool size of 2x2, which feeds into the second convolutional layer of 64 filters, a 3x3 kernel, and uses a ReLU activation function. The second max pooling layer is identical to the first max pooling layer. The third convolutional layer has 128 filters, a kernel size of 3x3, and employs a ReLU activation function. Again, the third max pooling layer is identical to the first two. The Flatten layer squishes the output of the previous layer into a 1-D vector. Next, a fully connected or dense layer of 512 units and a ReLU activation function feeds into a Dropout layer, which is a regularization technique that kills off a portion of the neurons at random during training, helping to prevent overfitting. The final layer is a dense layer with a single unit with a sigmoid activation function, which outputs a value in the range (0, 1) inclusive. Sigmoid is perfect for binary classification problems as we can map the output to one class if the output is over 0.5 and the other class if not.

In [ ]:
model1 = Sequential()

model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model1.add(MaxPooling2D((2, 2)))

model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))

model1.add(Conv2D(128, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))

model1.add(Flatten())

model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.5))

model1.add(Dense(1, activation='sigmoid'))

We will use Adam as our optimizer, binary crossentropy as our loss function, and accuracy as our standard evaluation metric, as well as AUC as an additional evaluation metric, since the final output submitted to the Kaggle competition will be judged on AUC.

Adam, or Adaptive Moment Estimation, is an algorithm that builds off of stochastic gradient descent (SGD) by adjusting the learning rate over time using the first and second moments of the gradient. Regular SGD can suffer from slow convergence or even divergence with a poorly chosen learning rate that remains fixed through the learning process. Adam's adaptive learning rate speeds up convergence time and reduces the chances of divergence as well. Adam was released to the general public in 2014 by Diederik Kingma, writing "The method is straightforward to implement, is computationally efficient, has little memory requirements, is invariant to diagonal rescaling of the gradients, and is well suited for problems that are large in terms of data and/or parameters." in Kingma et al., 2014. 

BCE loss measures the difference between true labels and the predicted pseudoprobabilities output by the final sigmoid layer in the CNN model. The loss is minimized when the predicted pseduoprobabilities are closest to the true labels. Therefore, BCE loss is commonly used in binary classification problems to train models with gradient descent optimization algorithms, such as Adam.

AUC, or "Area Under the ROC Curve" is a metric used to evaluate the performance of a binary classification model, although usually for datasets with imbalanced class sizes. Since our dataset has balanced classes, it is not particularly more useful than other available metrics like accuracy, but again it is part of the competition so we should include it to have a good yardstick of our leaderboard performance.

In [ ]:
model1.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Metrics.AUC()])

In [ ]:
history = model1.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.n//batch_size,
    epochs = num_epochs,
    validation_data = test_generator,
    validation_steps = test_generator.n//batch_size)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['AUC'])
plt.plot(history.history['val_AUC'])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Next, let's do some hyperparameter tuning using our old friend, grid search.

In [ ]:
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Define values to search
batch_sizes = [16, 32, 64]
epochs = [20, 30, 40]
learning_rates = [0.001, 0.003, 0.005]

# Define the model architecture
def RNN_model(learning_rate=0.005):
    model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(64, (3, 3), activation='relu'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Conv2D(128, (3, 3), activation='relu'))
    model1.add(MaxPooling2D((2, 2)))
    model1.add(Flatten())
    model1.add(Dense(512, activation='relu'))
    model1.add(Dropout(0.5))
    model1.add(Dense(1, activation='sigmoid'))
    
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create a KerasClassifier object to perform looping
model = KerasClassifier(build_fn=RNN_model, epochs=20, batch_size=32, verbose=0)
param_grid = dict(batch_size=batch_sizes, epochs=epochs, learning_rate=learning_rates)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Our best hyperparameter combination was a batch size of 32, over 40 training epochs, with a learning rate of 0.003, yielding an accuracy of ~ 87%.

**Model 2**

Let's see if we can get any better results using a simpler architecture, with far fewer layers, by cutting down to one convolutional layer and one max pooling layer. Remember, a more parsimonious model is always preferred to a more complex model given that they have the same explanatory power.

In [ ]:
model2 = Sequential()

model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())

model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Dense(1, activation='sigmoid'))

In [ ]:
# Define values to search
batch_sizes = [16, 32, 64]
epochs = [20, 30, 40]
learning_rates = [0.001, 0.003, 0.005]

# Define the model architecture
def RNN_model2(learning_rate=0.005):
    model2 = Sequential()
    model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
    model2.add(MaxPooling2D((2, 2)))
    model2.add(Flatten())
    model2.add(Dense(512, activation='relu'))
    model2.add(Dropout(0.5))
    model2.add(Dense(1, activation='sigmoid'))
    
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create a KerasClassifier object to perform looping
model2 = KerasClassifier(build_fn=RNN_model2, epochs=20, batch_size=32, verbose=0)
param_grid2 = dict(batch_size=batch_sizes, epochs=epochs, learning_rate=learning_rates)
grid2 = GridSearchCV(estimator=model, param_grid=param_grid2, cv=3, verbose=1)
grid_result2 = grid2.fit(X_train, y_train)

# Print the results
print(f"Best: {grid_result2.best_score_} using {grid_result2.best_params_}")

**Results + Analysis + Conclusion**

Our best hyperparameter combination was a batch size of 32, over 40 training epochs, with a learning rate of 0.005, yielding an accuracy of ~ 82%. We can see that while our results are still fairly good, we had better performance using the more complex model that was able to learn the separation slightly better. This makes sense because the more convolutional layers we have in the neural network, the more complex shapes and patterns they are able to learn as features. Histopathology images of cancer cells can be very difficult to parse as humans and may need several layers of convolution for sufficient deep learning. Higher amounts of epochs and slower learning rates were especially critical in model performance being as good as it was. With lower numbers of epochs the CNN lacked sufficient time to learn correctly and with a higher learning rate the model may fail to converge and overshoot the global minimum in the gradient descent.

CNN was an apt choice as an overarching deep learning technique, as it was specifically created to deal with classification of images, which is exactly the task set before us in this cancer cell detection challenge. To recap, our best model had four convolutional layers, three max pooling layers, and two dense layers. The first convolutional layer has 32 filters with a kernel size of 3x3 and a ReLU activation function. The input shape is height and width of the images (96x96) and 3 channels for RGB. Then our first max pooling layer has a pool size of 2x2, which feeds into the second convolutional layer of 64 filters, a 3x3 kernel, and uses a ReLU activation function. The second max pooling layer is identical to the first max pooling layer. The third convolutional layer has 128 filters, a kernel size of 3x3, and employs a ReLU activation function. Again, the third max pooling layer is identical to the first two. The Flatten layer squishes the output of the previous layer into a 1-D vector. Next, a fully connected or dense layer of 512 units and a ReLU activation function feeds into a Dropout layer, which is a regularization technique that kills off a portion of the neurons at random during training, helping to prevent overfitting. The final layer is a dense layer with a single unit with a sigmoid activation function, which outputs a value in the range (0, 1) inclusive. Sigmoid is perfect for binary classification problems as we can map the output to one class if the output is over 0.5 and the other class if not.

We learned that more complicated imagery like Gram stained cell histopathology microscope scans may require a more complex CNN than some of the more simplistic CNN tasks such as looking at faces or cats vs dogs, where there are a few main concepts to learn that are fairly obvious, such as eyes, mouth, nose, and tail. Cancerous cells can take on many different forms and the variations in cells can be amazingly complex even compared to the variations in human faces or animals. Our deeper CNN model managed to outperform our simpler model by a healthy margin due to its additional convolutional layers.

Potential enhancements for future endeavors could be adding in another regularization technique on top of dropout, which is weight decay, to prevent overfitting. We could try to implement transfer learning, where a pretrained model is used as a launching point for us to build on top of. Some examples of this are ResNet101, MobileNet, and Xception which are available to use from the Keras pretrained models. We could also build multiple CNN models and combine their outputs to form an ensemble learning network, which could also help reduce overfitting and generalize the performance of the model. Last but not least, we could always add more layers into our highest performing CNN and make the model even deeper, to see if the performs becomes any stronger.